Задачка будет стандартная - предсказать, вернет ли человек кредит. 
Описание данных можно посмотреть тут https://www.kaggle.com/ajay1735/hmeq-data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Загружаем данные и делаем простой препроцессинг. 
Убираем наны, категориальные фичи разбиваем в one-hot фичи. 

In [ ]:
data = pd.read_csv("../data/hmeq.csv")

In [ ]:
data.head()

Выкинием колонку в которой много нанов целиком

In [ ]:
data.drop("DEBTINC", axis = 1, inplace = True)

In [ ]:
data_clean = data.dropna()

In [ ]:
data_clean_one_hot = pd.get_dummies(data_clean, drop_first=True)

In [ ]:
data_clean_one_hot.head()

Наконец вытащим данные в два нампай массива x и y.

In [ ]:
x = data_clean_one_hot[data_clean_one_hot.columns[1:]].values
y = data_clean.BAD.values

## Попробуем решающее дерево

Проверьте, какой результат даст решающее дерево без настройки параметров на кросс валидации из 5-ти фолдов. Используйте KFold для кросс валидации

In [ ]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import roc_auc_score

In [ ]:
kfold = KFold(5)

y_true = []
predictions = []

for train_ind, test_ind in kfold.split(x):
    tree = DecisionTreeClassifier()
    
    tree.fit(x[train_ind], y[train_ind])
    predictions.extend(tree.predict(x[test_ind]))
    y_true.extend(y[test_ind])

In [ ]:
roc_auc_score(y_true, predictions)

## Используем sklearn для подбора параметров дерева по кросс валидации

Для настройки множества параметров модели можно использовать удобный класс из sklearn.model_selection - GridSearchCV. http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html . Это обертка над классификатором, куда надо засунуть классификатор, и питоновский словарь, в котором ключи - название параметров классификатора, которые будем настраивать, а значения - массивы всех значений которые нужно перебрать. 

GridSearchCV сделает кросс валидацию для всех комбинаций параметров, и выдаст статистику обучения и результаты работы алгоритма на всех сочетаниях параметров. Можно будет выбрать лучшее сочетание для обучения окончательной модели.

Ниже пример для настройки одного параметра.  

In [ ]:
from sklearn.model_selection import GridSearchCV

Попробуем подобрать критерий по которому происходит разбиение на каждом шагу. Это может быть информационная энтропия, как мы обсуждали, а может быть Gini критерий. Этот параметр называется criterion. Посмотрите какие значения надо подставить в этот параметр в документации.
http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier


Создайте словарь, с значениями, которые хотите перебрать в виде списка.

In [ ]:
criterion_search_dict = {'criterion' : * YOUR CODE HERE *}

Теперь создаем GridSearchCV. Первым параметром - классификатор, в param_grid отправляем наш словарь, в cv ставим кол-во фолдов для кросс валидации. return_train_score ставим True, чтобы получить результаты.

In [ ]:
grid_clf = GridSearchCV(tree, param_grid=criterion_search_dict, cv = 5, return_train_score=True)

Теперь фитим grid_clf во все данные, которые есть, кросс валидация происходит сама собой! 

In [ ]:
grid_clf.fit(x, y)

Полные результаты доступны в параметре .cv_results_

In [ ]:
grid_clf.cv_results_

Это словарь с кучей всякой инфы. Вытащить скоры из словаря можно по ключу 'mean_test_score'

In [ ]:
grid_clf.cv_results_['mean_test_score']

Посмотрите что еще ценного записано в cv_results_ чтобы знать.

Самый лучший классификатор можно вытащить из grid_clf:

In [ ]:
best_tree = grid_clf.best_estimator_

А важность фичей можно вытащить с помощью .feature_importances_. Какие фичи оказались важными? 
Можно поробовать выкинуть все остальные, и добавить попарные произведения этих. 

In [ ]:
best_tree.feature_importances_

## Повторите процедуру выше подобрав больше разных параметров.

In [ ]:
grid_search_dict = {'criterion' : * YOUR CODE HERE *, 'max_depth' : * YOUR CODE HERE *, ... * YOUR CODE HERE *}

In [ ]:
  * YOUR CODE HERE *

Какой набор параметров оказался самым лучшим? какой скор? 

## SVM

Попробуйте обучить svm классификатор. Поисследуйте, какие параметры можно настраивать там.


http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

## KNN

Аналогично для KNN

http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier

Кто всех порвал?) 